In [1]:
from transformers import pipeline
import torch
import torch.nn.functional as F

In [2]:
classifier = pipeline("sentiment-analysis")

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [3]:
res = classifier("We are very happy to show you the HuggingFace Transformers library.")

In [4]:
print(res)

[{'label': 'POSITIVE', 'score': 0.9997598528862}]


In [5]:
results = classifier(["We are very happy to show you the HuggingFace Transformers library.",
                  "We hope you don't hate it."])

In [6]:
for result in results:
    print(result)

{'label': 'POSITIVE', 'score': 0.9997598528862}
{'label': 'NEGATIVE', 'score': 0.530862033367157}


In [7]:
# "sentiment-analysis"의 기본 함수는 아래 모델과 같이 "model_name"에 지정된 함수 이름입니다.
# 나중에 우리는 이 모델 이름을 바꿔가면서 사용할 겁니다.
model_name = "distilbert-base-uncased-finetuned-sst-2-english"

In [8]:
# 지금은 model_name에 전달한 인수가 기본값과 같기 때문에 아까와 결과는 동일할 겁니다.
classifier = pipeline("sentiment-analysis", model=model_name)

In [9]:
results = classifier(["We are very happy to show you the HuggingFace Transformers library.",
                  "We hope you don't hate it."])

for result in results:
    print(result)

{'label': 'POSITIVE', 'score': 0.9997598528862}
{'label': 'NEGATIVE', 'score': 0.530862033367157}


In [10]:
# 텐서플로우를 사용할 경우 아래와 같이 바꿔서 실행하세요.
# from transformers import AutoTokenizer, TFAutoModelForSequenceClassification
# 여전히, 이 모델과 토크나이저는 pipeline 객체의 기본값입니다.
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [11]:
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [12]:
model_name = "distilbert-base-uncased-finetuned-sst-2-english"

In [13]:
classifier = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

In [14]:
results = classifier(["We are very happy to show you the HuggingFace Transformers library.",
                  "We hope you don't hate it."])

for result in results:
    print(result)

{'label': 'POSITIVE', 'score': 0.9997598528862}
{'label': 'NEGATIVE', 'score': 0.530862033367157}


In [15]:
tokens = tokenizer.tokenize("We are very happy to show you the HuggingFace Transformers library.")

In [16]:
token_ids = tokenizer.convert_tokens_to_ids(tokens)

In [17]:
input_ids = tokenizer("We are very happy to show you the HuggingFace Transformers library.")

In [18]:
print(f"    Tokens: {tokens}")
print(f"Token IDs: {token_ids}")
print(f"Input IDs: {input_ids}")

    Tokens: ['we', 'are', 'very', 'happy', 'to', 'show', 'you', 'the', 'hugging', '##face', 'transformers', 'library', '.']
Token IDs: [2057, 2024, 2200, 3407, 2000, 2265, 2017, 1996, 17662, 12172, 19081, 3075, 1012]
Input IDs: {'input_ids': [101, 2057, 2024, 2200, 3407, 2000, 2265, 2017, 1996, 17662, 12172, 19081, 3075, 1012, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


위 결과에서 `input_ids`의 시작과 끝에 101과 102가 추가로 붙어 있습니다. 이는 단순히 각각 문장의 시작과 끝을 의미할 뿐인 숫자라 크게 주목할 필요는 없습니다.

In [19]:
X_train = ["We are very happy to show you the HuggingFace Transformers library.",
           "We hope you don't hate it."]

In [20]:
# 텐서플로우 사용시 return_tensors 인수를 "tf"로 지정해주세요.
batch = tokenizer(X_train, padding=True, truncation=True, max_length=512, return_tensors="pt")

In [21]:
print(batch)

{'input_ids': tensor([[  101,  2057,  2024,  2200,  3407,  2000,  2265,  2017,  1996, 17662,
         12172, 19081,  3075,  1012,   102],
        [  101,  2057,  3246,  2017,  2123,  1005,  1056,  5223,  2009,  1012,
           102,     0,     0,     0,     0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0]])}


훈련 데이터를 `tokenizer`로 변환하면 각각의 문장을 토크나이즈한 결과를 텐서 형태로 변환합니다. 그 결과는 `'input_ids'`라는 키에 대한 값으로 할당됩니다. 또한, `'attention_mask'`라는 이름의 텐서 쌍도 할당되는데 이게 뭘 뜻하는지는 나중에 알아보도록 하겠습니다.

In [23]:
with torch.no_grad():
    outputs = model(**batch) ## 텐서플로우에서 파이토치의 no_grad()에 해당하는 함수를 이용할 경우 model(batch)라고만 입력해도 코드가 작동합니다.
    print(outputs)
    
    # 긍정일 확률과 부정일 확률 계산(softmax 함수 이용)
    predictions = F.softmax(outputs.logits, dim=1)
    print(predictions)
    
    # 계산된 확률 중 더 높은 쪽 확률의 인덱스 반환
    labels = torch.argmax(predictions, dim=1)
    print(labels)
    
    # 인덱스를 사람이 알아볼 수 있는 실제 레이블('POSITIVE', 'NEGATIVE')로 변환
    labels = [model.config.id2label[label_id] for label_id in labels.tolist()]
    print(labels)

SequenceClassifierOutput(loss=None, logits=tensor([[-4.0584,  4.2758],
        [ 0.0818, -0.0418]]), hidden_states=None, attentions=None)
tensor([[2.4012e-04, 9.9976e-01],
        [5.3086e-01, 4.6914e-01]])
tensor([1, 0])
['POSITIVE', 'NEGATIVE']


In [24]:
with torch.no_grad():
    # loss 계산을 위해 labels 인수에 값 부여
    outputs = model(**batch, labels=torch.tensor([1, 0]))
    print(outputs)
    
    predictions = F.softmax(outputs.logits, dim=1)
    print(predictions)
    
    labels = torch.argmax(predictions, dim=1)
    print(labels)
    
    labels = [model.config.id2label[label_id] for label_id in labels.tolist()]
    print(labels)

SequenceClassifierOutput(loss=tensor(0.3168), logits=tensor([[-4.0584,  4.2758],
        [ 0.0818, -0.0418]]), hidden_states=None, attentions=None)
tensor([[2.4012e-04, 9.9976e-01],
        [5.3086e-01, 4.6914e-01]])
tensor([1, 0])
['POSITIVE', 'NEGATIVE']


여기까지의 과정에서 파인 튜닝을 진행하지는 않았지만 *일단 진행했다고 가정하고* 이 모델을 저장해보도록 하겠습니다.

In [27]:
import os
from os import path

save_directory = "./saved"
if not path.isdir(save_directory):
    os.makedirs(save_directory)

In [28]:
tokenizer.save_pretrained(save_directory)
model.save_pretrained(save_directory)

In [29]:
# 모델 불러올 때
tokenizer = AutoTokenizer.from_pretrained(save_directory)
model = AutoModelForSequenceClassification.from_pretrained(save_directory)

## 허깅페이스 모델 허브
허깅페이스에서 다른 모델을 불러올 때의 과정을 실습합니다.

In [30]:
model_name = "oliverguhr/german-sentiment-bert"

In [31]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

c:\Users\tjk\AppData\Local\anaconda3\envs\dp_final\lib\site-packages\huggingface_hub\file_download.py:137: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\tjk\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [35]:
X_train_german = ["Mit keinem guten Ergebnis", "Das war unfair", "Das ist gar nicht mal so gut", # 부정
         "nicht so schlecht wie erwartet", "Das war gut!", # 긍정
         "Sie fährt ein grünes Auto."] # 중립

In [36]:
batch = tokenizer(X_train_german, padding=True, truncation=True, max_length=512, return_tensors="pt")
print(batch)

{'input_ids': tensor([[    3,   304,  8524,  5569,  2011,     4,     0,     0,     0],
        [    3,   295,   185,   174,  8716,   124,     4,     0,     0],
        [    3,   295,   127,  2523,   149,  2723,   181,  1522,     4],
        [    3,   149,   181,  6975,   246,  6303,     4,     0,     0],
        [    3,   295,   185,  1522, 26982,     4,     0,     0,     0],
        [    3,   371,  9755,    39, 19044, 26902,  3512, 26914,     4]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 0, 0],
        [1, 1, 1, 1, 1, 1, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1]])}


In [37]:
with torch.no_grad():
    outputs = model(**batch)
    label_ids = torch.argmax(outputs.logits, dim=1)
    print(label_ids)
    
    labels = [model.config.id2label[label_id] for label_id in label_ids.tolist()]
    print(labels)

tensor([1, 1, 1, 0, 0, 2])
['negative', 'negative', 'negative', 'positive', 'positive', 'neutral']


여기서 `batch = tokenizer(X_train_german, padding=True, truncation=True, max_length=512)`와 같이 텐서를 지정하지 않으면 결과는 리스트로 반환됩니다. 이 때는 `with` 구문으로 들어가기 전에 한 가지 과정을 더 거친 후, `with` 문 내에서도 구문을 조금 수정해야 합니다.  
```Python
batch = torch.tensor(batch["input_ids"])

with torch.no_grad():
    outputs = model(batch) ## **batch를 batch로 수정합니다.
    label_ids = torch.argmax(outputs.logits, dim=1)
    print(label_ids)
    
    labels = [model.config.id2label[label_id] for label_id in label_ids.tolist()]
    print(labels)
```